<a href="https://colab.research.google.com/github/tiendatngcs/NLP/blob/master/Vietnamese_news_topic_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pickle as pkl
import tqdm

## Gathering data

We scrape the web with BeautifulSoup



Prototype:

In [3]:
url = 'https://vnexpress.net/hang-khong-hut-nguon-thu-cao-diem-he-4152498.html'

In [4]:
markup = requests.get(url).text
soup = BeautifulSoup(markup=markup, features="lxml")

In [5]:
text = [p.text for p in soup.find(name="section", class_="section page-detail top-detail").find(name="article", class_="fck_detail ").find_all("p", class_="Normal")]

In [6]:
text

['Sau khi dịch bệnh được kiểm soát hồi cuối tháng 4, hàng không Việt dần tăng khai thác trở lại các chặng nội địa từ tháng 5. Tiếp đà phục hồi nhanh chóng, các hãng rất kỳ vọng sẽ bù đắp được dòng tiền nhờ cao điểm hè từ tháng 7 đến tháng 9. Tuy nhiên, làn sóng dịch thứ hai bùng phát từ cuối tháng 7 đã chuyển giai đoạn cao điểm thành thấp điểm.',
 'Theo thống kê của Cục Hàng không giai đoạn 19/7 – 18/8, tổng số chuyến bay của 5 hãng trong nước chỉ còn hơn 16.400, giảm 33% so với tháng trước và 45% so với cùng kỳ năm ngoái. Trong đó, Vietnam Airlines bị giảm hơn 4.300 chuyến, tương đương giảm gần 40% so với tháng 7, còn khoảng 6.700 chuyến. Vietjet Air khai thác hơn 5.700 chuyến, giảm 31%, tương đương trên 2.600 chuyến bay.',
 'Tổng số chuyến của các hãng Bamboo Airways, Pacific Airways, VASCO giảm lần lượt giảm 17%, 15% và 43% so với giai đoạn tháng 7. Các hãng bay đã hủy 490 chuyến, tăng 325 chuyến so với tháng trước đó. Các hãng gần đây của cũng phải giảm tần suất hoặc dừng khai thác

Gather article urls

In [2]:
topic_list = [
              "business",
              "entertainment",
              "sports",
              "laws",
              "education",
              "health",
              "life",
              "travel",
              "science",
              "digital",
              "vehicles",
              "comedy",
]
topic_dict = dict(enumerate(topic_list))

In [3]:
topic_urls = [
              "https://vnexpress.net/kinh-doanh", # business
              "https://vnexpress.net/giai-tri", # entertainment
              "https://vnexpress.net/the-thao", # sports
              "https://vnexpress.net/phap-luat", # laws
              "https://vnexpress.net/giao-duc", # education
              "https://vnexpress.net/suc-khoe", # health
              "https://vnexpress.net/doi-song", # life
              "https://vnexpress.net/du-lich", # travel
              "https://vnexpress.net/khoa-hoc", # science
              "https://vnexpress.net/so-hoa", # digital
              "https://vnexpress.net/oto-xe-may", # vehicles
              "https://vnexpress.net/hai"] # comedy
pages = [""] + ["/p"+ str(i) for i in range(2, 20)]

In [6]:
def get_articles_links(topic_link:str):
  markup = requests.get(topic_link).text
  soup = BeautifulSoup(markup=markup, features="lxml")
  
  # article_tags = soup.find_all("article", class_="item-news item-news-common ")
  # if len(article_tags) == 0:
  #   article_tags = soup.find_all("article", class_="item-news item-news-common")
  # if len(article_tags) == 0:
  #   return []
  # urls = [article_tag.find("a", href=True)["href"] for article_tag in article_tags]
  urls = []
  for tag in soup.find_all("article"):
    a_tag = tag.find("a", href=True)
    if a_tag is not None:
      urls.append(a_tag["href"])
  # urls = vne_soup.find("article", class_="item-news item news-common").find_all(a, href=True)
  return urls

In [ ]:
# for topic_url in topic_urls:
#   print(topic_url)
#   get_articles_links(topic_url)

https://vnexpress.net/kinh-doanh
https://vnexpress.net/giai-tri
https://vnexpress.net/the-thao
https://vnexpress.net/phap-luat
https://vnexpress.net/giao-duc
https://vnexpress.net/suc-khoe
https://vnexpress.net/doi-song
https://vnexpress.net/du-lich
https://vnexpress.net/khoa-hoc
https://vnexpress.net/so-hoa
https://vnexpress.net/oto-xe-may
https://vnexpress.net/hai


In [7]:
articles = {}
for i, topic_url in enumerate(topic_urls):
  articles[i] = []
  for page in pages:
    url = topic_url + page
    print(url)
    articles[i] = articles[i] + get_articles_links(url)


https://vnexpress.net/kinh-doanh
https://vnexpress.net/kinh-doanh/p2
https://vnexpress.net/kinh-doanh/p3
https://vnexpress.net/kinh-doanh/p4
https://vnexpress.net/kinh-doanh/p5
https://vnexpress.net/kinh-doanh/p6
https://vnexpress.net/kinh-doanh/p7
https://vnexpress.net/kinh-doanh/p8
https://vnexpress.net/kinh-doanh/p9
https://vnexpress.net/kinh-doanh/p10
https://vnexpress.net/kinh-doanh/p11
https://vnexpress.net/kinh-doanh/p12
https://vnexpress.net/kinh-doanh/p13
https://vnexpress.net/kinh-doanh/p14
https://vnexpress.net/kinh-doanh/p15
https://vnexpress.net/kinh-doanh/p16
https://vnexpress.net/kinh-doanh/p17
https://vnexpress.net/kinh-doanh/p18
https://vnexpress.net/kinh-doanh/p19
https://vnexpress.net/giai-tri
https://vnexpress.net/giai-tri/p2
https://vnexpress.net/giai-tri/p3
https://vnexpress.net/giai-tri/p4
https://vnexpress.net/giai-tri/p5
https://vnexpress.net/giai-tri/p6
https://vnexpress.net/giai-tri/p7
https://vnexpress.net/giai-tri/p8
https://vnexpress.net/giai-tri/p9
https:

Save the article links

In [8]:
with open("/content/drive/My Drive/articles.pkl", "wb") as outfile:
  pkl.dump(articles, outfile, pkl.HIGHEST_PROTOCOL)

Load the article links

In [9]:
with open("/content/drive/My Drive/articles.pkl", "rb") as infile:
  load_articles = pkl.load(infile)

In [10]:
for (key, value) in load_articles.items():
  print("{}: {}".format(key, len(value)))

0: 585
1: 584
2: 580
3: 152
4: 143
5: 587
6: 575
7: 575
8: 143
9: 583
10: 156
11: 143


In [11]:
def url_to_corpus(url:str):
  markup = requests.get(url).text
  soup = BeautifulSoup(markup=markup, features="lxml")
  # text = [p.text for p in soup.find(name="section", class_="section page-detail top-detail").find(name="article", class_="fck_detail ").find_all("p", class_="Normal")]

  text = [p.text for p in soup.find(name="section", class_="section page-detail top-detail").find(name="article").find_all("p")]
  return " ".join(text)

In [12]:
url_to_corpus(load_articles[3][2])

'Ngày 28/8, Nguyễn Văn Thiết 29 tuổi, xã Quảng Văn, huyện Quảng Xương, một trong những người tham gia nhóm bắt giữ, đã bị Công an huyện Quảng Xương tạm giữ để làm rõ hành vi bắt cóc nhằm chiếm đoạt tài sản. Công an đang truy tìm Nguyễn Khắc Ngọc, 36 tuổi, trú xã Quảng Thịnh, thành phố Thanh Hóa; Lê Huy Lợi, 30 tuổi, ở xã Quảng Phúc, huyện Quảng Xương và người tên Tiện. Nghi can Nguyễn Văn Thiết bị bắt. Ảnh: Lam Sơn. Công an xác định, khoảng tháng 2, anh Tân vay Ngọc 66 triệu đồng, tính cả gốc và lãi khoản này lên đến 90 triệu đồng. Khi anh Tân chưa có tiền trả, Ngọc lên kế hoạch bắt người để đòi nợ. Khoảng 1h ngày 26/8, Ngọc cùng ba nghi phạm trèo qua tường rào, lẻn vào nhà anh Tân ở xã Quảng Hợp, dùng dao uy hiếp, ép anh lên xe, đưa đến nhiều nơi ở huyện Quảng Xương. Nhóm này đánh anh Tân, yêu cầu gọi điện thoại cho người thân bảo thu xếp 90 triệu đồng. Khi tiền trao, người mới được thả. Tuy nhiên đến 4h cùng ngày, biết công an đang truy tìm, nhóm nghi phạm chở nạn nhân quay lại xã Qu

In [13]:
corpus_articles_by_topic_dict = {}
for (key, value) in load_articles.items():
  print(key)
  if None in value:
    print("error")
  corpus_articles_by_topic_dict[key] = []
  for url in value:
    try:
      corpus_articles_by_topic_dict[key].append(url_to_corpus(url))
    except:
      print(url)

0
https://vnexpress.net/chung-khoan-mat-moc-880-diem-4153551.html
https://vnexpress.net/toa-dam-xu-huong-ung-dung-ai-trong-ngan-hang-viet-4153141.html
https://vnexpress.net/hon-mot-thap-ky-theo-duoi-khat-vong-sang-tao-cua-kdi-holdings-4151439.html
https://vnexpress.net/fpt-van-hanh-so/doanh-nghiep-tang-loi-nhuan-nho-ai-4151404.html
https://vnexpress.net/khong-gian-song-da-the-he-chieu-long-gia-dinh-thoi-hien-dai-4151156.html
https://vnexpress.net/fpt-van-hanh-so/vi-sao-ai-thuc-day-su-bung-no-thuong-mai-dien-tu-dong-nam-a-4146273.html
https://vnexpress.net/8-trieu-phu-ty-phu-co-thoi-quen-chi-tieu-ky-la-4150826.html
https://vnexpress.net/fpt-van-hanh-so/sep-cac-doanh-nghiep-hang-dau-hien-ke-vuot-khung-hoang-covid-19-4150417.html
https://vnexpress.net/kien-truc-thap-doi-cung-mang-xanh-da-tang-tai-sunshine-horizon-4148041.html
https://vnexpress.net/elon-musk-kiem-va-tieu-tien-nhu-the-nao-4149191.html
https://vnexpress.net/fpt-van-hanh-so/loi-giai-cong-nghe-giup-doanh-nghiep-vuot-khung-hoan

In [14]:
with open("/content/drive/My Drive/corpus.pkl", "wb") as outfile:
  pkl.dump(corpus_articles_by_topic_dict, outfile, pkl.HIGHEST_PROTOCOL)

The number of articles for each topic vairies. Let's assume the articles across all topics have roughly similar amount of content (same string length)
Let's pick 140 articles from each topic for training. Total training size = 140 * 11 = 1540 articles.

In [16]:
training_articles = {}
for (key, value) in load_articles.items():
  training_articles[key] = value[:140]

In [ ]:
vne = "https://vnexpress.net/"
vne_page = requests.get(vne).text
vne_soup = BeautifulSoup(markup=vne_page, features="lxml")
vne_soup_menu = vne_soup.find_all("section", class_="wrap-all-menu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
vne_soup_menu

[<section class="wrap-all-menu"></section>]